## Multiple Chain 快速入门

Runnables 可以轻松地用来串联多个 Chains，使用 RunnablePassthrough 将输出同时传给多条后继链。

```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

本指南展示如何使用 Runnable 实现多个 AI 关于相同话题的辩论：

```
    输入话题
       |
       |
    原始观点
      / |\
     /  |  \
 正面论述| 反面论述
     \  | /
      \ |/
     最终总结
```

In [1]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 创建正面论证的处理链，列出关于基础回应的正面或有利的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的正面或有利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的反面或不利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出批评后生成最终回应"),
        ]
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [2]:
print(chain.invoke({"input": "房地产低迷"}))

在分析房地产市场的低迷时，我们可以看到各种正面和反面的观点。虽然这些观点各有其合理性，但也存在一些需要批评和进一步思考的地方。

### 批评观点

1. **购房机会增加的局限性**：虽然低迷期间房价下跌为购房者提供了机会，但对于许多首次购房者而言，经济不景气可能导致收入不稳定，反而使得购房能力受到限制。此外，许多人可能因为对未来经济前景的不确定性而选择观望，而不是立即购房。

2. **市场调整的风险**：市场的必要调整虽然可以淘汰不符合需求的项目，但在此过程中，可能会导致大量失业和社会动荡。市场的自我调整并不是总能顺利进行，可能会引发更深层次的经济问题。

3. **租赁市场的饱和**：虽然租赁市场可能因购房意愿下降而发展，但快速增长的租赁需求也可能导致租金上涨，给低收入家庭带来更大的负担。此外，租赁市场的繁荣并不一定能替代稳定的购房需求。

4. **政策调整的局限性**：尽管政府可能会采取措施来刺激市场，但政策的实施效果往往滞后，且可能受到政治和经济环境的影响。政策调整可能无法及时满足市场的需求，导致恢复缓慢。

5. **技术创新的障碍**：虽然技术进步能够提升市场效率，但并不是所有开发商和投资者都有能力或意愿去投资新技术。小型企业或地方开发商可能面临技术和资金的双重压力。

6. **人口流动的矛盾**：虽然某些地区可能因人口流入而受益，但整体人口老龄化和迁移趋势可能导致一些地区继续面临萎缩。这种不均衡的现象可能加剧地区差异，造成社会问题。

7. **可持续发展意识的真正影响**：尽管对可持续发展的关注在增加，但如何将这种意识转化为实际的市场需求仍然是一个挑战。绿色建筑的成本可能较高，且并非所有购房者都愿意为此支付额外费用。

### 最终回应

总的来说，尽管房地产市场的低迷确实带来了某些机会和优势，但也伴随着许多风险和挑战。我们在分析市场动态时，不能仅仅关注表面的积极因素，还需深入挖掘潜在的负面影响和局限性。只有通过全面的分析，我们才能更好地理解市场现状，并为未来的决策提供有价值的参考。同时，政府、开发商和消费者也应共同努力，以应对低迷带来的挑战，推动房地产市场的健康发展。


#### 流式输出

In [3]:
## chain 最终输出经过了 StrOutputParser 处理，所以可以直接输出流式输出 s
for s in chain.stream({"input": "全球经济"}):
    print(s, end="", flush=True)

在探讨全球经济的各个方面时，正面观点和反面观点的对比有助于更全面地理解其复杂性和动态变化。以下是对正面和反面观点的最终回应：

1. **全球化的利与弊**：
   - **最终回应**：全球化的确带来了贸易便利和技术传播，但它也暴露了国际依存关系的脆弱性。在应对未来的全球性挑战时，各国需要在开放与自给自足之间找到平衡，以降低对外部环境的脆弱性。

2. **数字经济的崛起**：
   - **最终回应**：数字经济为全球经济带来了巨大的创新潜力和效率提升，但在享受这些红利的同时，政策制定者应关注数字鸿沟问题。通过加强教育和基础设施投资，确保更多人能够参与数字经济，是实现包容性增长的重要一步。

3. **气候变化与经济可持续性**：
   - **最终回应**：虽然向绿色经济转型面临挑战，但这一过程也为创新和投资提供了机会。各国应通过国际合作和技术共享来缓解转型带来的负担，从而实现经济增长与环境保护的双赢。

4. **国际贸易政策的影响**：
   - **最终回应**：合理的国际贸易政策能够促进经济合作，但过度的保护主义可能会适得其反。各国应致力于通过多边合作解决贸易摩擦，以构建更加稳定和可预见的国际贸易环境。

5. **技术进步与劳动力市场**：
   - **最终回应**：技术进步虽然会淘汰一些传统岗位，但同时也创造了新的就业机会。为了减轻转型带来的负面影响，政府和企业需要共同努力，提供职业培训和再就业支持，帮助劳动力适应变化。

6. **新兴市场的崛起**：
   - **最终回应**：新兴市场的快速发展为全球经济注入了活力，但其脆弱性也不可忽视。国际社会应关注这些市场的稳定性，通过投资和支持政策，助力其可持续发展。

综上所述，全球经济的发展既面临机遇也存在挑战。通过综合考虑各方面的观点，政策制定者和利益相关者可以更好地应对复杂的全球经济环境，从而实现可持续增长和共同繁荣。只有在深入分析和理解这些问题的基础上，我们才能制定出有效的策略，以应对未来的挑战。

### Homework: 实现一个多链版本的代码生成，输入功能需求，输出2种（Python，Java）以上编程语言的代码实现。